In [4]:
import os
import sys
import numpy as np
import pyscipopt.scip as sp
import multiprocessing as md
from pathlib import Path 
from functools import partial
from node_selectors import OracleNodeSelectorAbdel
from recorders import LPFeatureRecorder, CompFeaturizer


%load_ext autoreload
%autoreload 2


In [27]:
class OracleNodeSelRecorder(OracleNodeSelectorAbdel):
    
    def __init__(self, oracle_type, comp_behaviour_saver=None):
        super().__init__(oracle_type)
        self.counter = 0
        self.comp_behaviour_saver = comp_behaviour_saver
    
    def set_LP_feature_recorder(self, LP_feature_recorder):
        self.comp_behaviour_saver.set_LP_feature_recorder(LP_feature_recorder)
        self.counter = 0
        
        
    def nodecomp(self, node1, node2):
        comp_res, comp_type = super().nodecomp(node1, node2, return_type=True)
        
        if comp_type in [-1,1]:
            self.comp_behaviour_saver.save_comp(self.model, 
                                                node1, 
                                                node2,
                                                comp_res,
                                                self.counter) 
        
            print("saved comp # " + str(self.counter))
            self.counter += 1
        
        #make it shit to generate more data !
        if comp_type in [-1,1]:
            comp_res = -1 if comp_res == 1 else 1
        else:
            comp_res = 0
            
        return comp_res


In [28]:
def run_episode(oracle_type, instance,  save_dir):
    
    model = sp.Model()
    model.hideOutput()
    
   

    model.setParam('constraints/linear/upgrade/logicor', 0)
    model.setParam('constraints/linear/upgrade/indicator',0)
    model.setParam('constraints/linear/upgrade/knapsack', 0)
    model.setParam('constraints/linear/upgrade/setppc', 0)
    model.setParam('constraints/linear/upgrade/xor', 0)
    model.setParam('constraints/linear/upgrade/varbound', 0)
    
    
    #Setting up oracle selector
    instance = str(instance)
    model.readProblem(instance)
    
    optsol = model.readSolFile(instance.replace(".lp", ".sol"))
    comp_behaviour_saver = CompFeaturizer(f"{save_dir}", 
                                              instance_name=str(instance).split("/")[-1])
    oracle_ns = OracleNodeSelRecorder(oracle_type, comp_behaviour_saver)
    oracle_ns.setOptsol(optsol)
    oracle_ns.set_LP_feature_recorder(LPFeatureRecorder(model,
                                                        'cpu'))
    
    model.includeNodesel(oracle_ns, "oracle_recorder", "testing",
                         536870911,  536870911)

    #print(f"Getting behaviour for instance {problem} "+ str(instance).split("/")[-1] )

    # Run the optimizer
    model.freeTransform()
    model.readProblem(instance)
    model.optimize()
    with open("nnodes.csv", "a+") as f:
        f.write(f"{model.getNNodes()},")
        f.close()
    with open("times.csv", "a+") as f:
        f.write(f"{model.getSolvingTime()},")
        f.close()
        
    return 1


def run_episodes(oracle_type, instances, save_dir):
    
    for instance in instances:
        run_episode(oracle_type, instance, save_dir)
        
    print("finished running episodes for process " + str(md.current_process()))
        
    return 1


In [23]:
def distribute(n_instance, n_cpu):
    if n_cpu == 1:
        return [(0, n_instance)]
    
    k = n_instance //( n_cpu -1 )
    r = n_instance % (n_cpu - 1 )
    res = []
    for i in range(n_cpu -1):
        res.append( ((k*i), (k*(i+1))) )
    
    res.append(((n_cpu - 1) *k ,(n_cpu - 1) *k + r ))
    return res


In [1]:
oracle = 'optimal_plunger'
problem = 'FCMCNF'
data_partitions = ['train', 'valid']
n_cpu = 4


In [29]:
with open("nnodes.csv", "w") as f:
    f.write("")
    f.close()
with open("times.csv", "w") as f:
    f.write("")
    f.close()


#Initializing the model 
for i in range(1, len(sys.argv), 2):
    if sys.argv[i] == '-oracle':
        oracle = str(sys.argv[i + 1])
    if sys.argv[i] == '-problem':
        problem = str(sys.argv[i + 1])
    if sys.argv[i] == '-n_cpu':
        n_cpu = int(sys.argv[i + 1])


for data_partition in data_partitions:

    save_dir = lp_dir= os.path.join(os.path.abspath(''), f"./data/{problem}/{data_partition}")

    try:
        os.makedirs(save_dir)
    except FileExistsError:
        ""


    instances = list(Path(os.path.join(os.path.abspath(''), 
                                       f"../problem_generation/data/{problem}/{data_partition}")).glob("*.lp"))
    n_instance = len(instances)

    print(f"Geneating {data_partition} samples from {n_instance} instances using oracle {oracle}")

    chunck_size = int(np.floor(len(instances)/n_cpu))
    processes = [  md.Process(name=f"worker {p}", 
                                    target=partial(run_episodes,
                                                    oracle_type=oracle,
                                                    instances=instances[ p1 : p2], 
                                                    save_dir=save_dir))
                    for p,(p1,p2) in enumerate(distribute(n_instance, n_cpu))]



    a = list(map(lambda p: p.start(), processes)) #run processes
    b = list(map(lambda p: p.join(), processes)) #join processes


nnodes = np.genfromtxt("nnodes.csv", delimiter=",")[:-1]
times = np.genfromtxt("times.csv", delimiter=",")[:-1]

print(f"Mean number of node created  {np.mean(nnodes)}")
print(f"Mean solving time  {np.mean(times)}")
print(f"Median number of node created  {np.median(nnodes)}")
print(f"Median solving time  {np.median(times)}")

    

Geneating train samples from 622 instances using oracle optimal_plunger
saved comp # 0
saved comp # 1
saved comp # 2
saved comp # 3
saved comp # 4
saved comp # 0
saved comp # 1
saved comp # 5
saved comp # 0
saved comp # 1
saved comp # 0
saved comp # 2
saved comp # 3
saved comp # 4
saved comp # 6
saved comp # 1
saved comp # 2


Warning: ('coefficients not available for constraints of type ', 'logicor')

Exception ignored in: 'pyscipopt.scip.PyNodeselComp'
Traceback (most recent call last):
  File "/tmp/ipykernel_112846/854818619.py", line 17, in nodecomp
  File "/local_workspace/labaabde/learn2selectnodes/node_selection/recorders.py", line 79, in save_comp
    torch_geometric_data = self.get_torch_geometric_data(model, node1, node2, comp_res)
  File "/local_workspace/labaabde/learn2selectnodes/node_selection/recorders.py", line 87, in get_torch_geometric_data
    triplet = self.get_triplet_tensors(model, node1, node2, comp_res)
  File "/local_workspace/labaabde/learn2selectnodes/node_selection/recorders.py", line 141, in get_triplet_tensors
    self.LP_feature_recorder.record_sub_milp_graph(model, node1)
  File "/local_workspace/labaabde/learn2selectnodes/node_selection/recorders.py", line 296, in record_sub_milp_graph
    graph = self.get_graph(model, parent).copy()
  File "/local_workspace/labaabde/learn2selectnodes/node_selection/recorders.py", line 283, in get_graph
    self.recor

saved comp # 3


Warning: ('coefficients not available for constraints of type ', 'logicor')

...

Exception ignored in: 'pyscipopt.scip.PyNodeselComp'
Traceback (most recent call last):
  File "/tmp/ipykernel_112846/854818619.py", line 17, in nodecomp


saved comp # 70

  File "/local_workspace/labaabde/learn2selectnodes/node_selection/recorders.py", line 79, in save_comp


    torch_geometric_data = self.get_torch_geometric_data(model, node1, node2, comp_res)
  File "/local_workspace/labaabde/learn2selectnodes/node_selection/recorders.py", line 87, in get_torch_geometric_data
    triplet = self.get_triplet_tensors(model, node1, node2, comp_res)
  File "/local_workspace/labaabde/learn2selectnodes/node_selection/recorders.py", line 142, in get_triplet_tensors
    self.LP_feature_recorder.record_sub_milp_graph(model, node2)
  File "/local_workspace/labaabde/learn2selectnodes/node_selection/recorders.py", line 296, in record_sub_milp_graph
    graph = self.get_graph(model, parent).copy()
  File "/local_workspace/labaabde/learn2selectnodes/node_selection/recorders.py", line 283, in get_graph
    self.record_sub_milp_graph(model, sub_milp)
  File "/local_workspace/labaabde/learn2selectnodes/node_selection/recorders.py", line 296, in record_sub_milp_graph
    graph = self.get_graph(model, parent).copy()
  File "/local_workspace/labaabde/learn2selectnodes/node_s

saved comp # 44
saved comp # 45
saved comp # 46
saved comp # 26
saved comp # 71
saved comp # 72
saved comp # 73
saved comp # 74
saved comp # 75
saved comp # 76
saved comp # 77
saved comp # 78
saved comp # 79
saved comp # 80
saved comp # 81
saved comp # 82
saved comp # 83
saved comp # 84
saved comp # 85
saved comp # 86
saved comp # 87
saved comp # 88
saved comp # 89
saved comp # 27
saved comp # 90
saved comp # 91
saved comp # 47
saved comp # 92saved comp # 48

saved comp # 93saved comp # 49

saved comp # 50
saved comp # 51
saved comp # 52
saved comp # 53
saved comp # 54
saved comp # 28
saved comp # 29
saved comp # 30
saved comp # 94
saved comp # 95
saved comp # 96
saved comp # 97
saved comp # 98
saved comp # 99
saved comp # 100
saved comp # 101
saved comp # 102
saved comp # 103
saved comp # 104
saved comp # 105
saved comp # 106
saved comp # 107
saved comp # 31
saved comp # 32
saved comp # 33
saved comp # 108saved comp # 34

saved comp # 109
saved comp # 110
saved comp # 35
saved comp # 